<p style="font-weight:bold;"> <span style="font-size: 36px"> Test DataNode Validation </span> </p>

In [0]:
#!import "../Import/Importers"

In [0]:
#!import "TestData"

# Workspace Initialization 

In [0]:
await Import.FromString(novelties).WithType<Novelty>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocTypes).WithType<AocType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(canonicalAocConfig).WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync();

In [0]:
await DataSource.UpdateAsync(reportingNodes);
await DataSource.UpdateAsync<Portfolio>(dt1.RepeatOnce());
await DataSource.UpdateAsync<Portfolio>(dtr1.RepeatOnce());
await DataSource.UpdateAsync<GroupOfInsuranceContract>(new [] {dt11});
await DataSource.UpdateAsync<GroupOfReinsuranceContract>(new [] {dtr11});

In [0]:
await DataSource.UpdateAsync(new [ ] {dt11State,dtr11State});

In [0]:
await Import.FromString(estimateType).WithType<EstimateType>().WithTarget(DataSource).ExecuteAsync();
await Import.FromString(economicBasis).WithType<EconomicBasis>().WithTarget(DataSource).ExecuteAsync();

In [0]:
await DataSource.UpdateAsync(new [ ] {yieldCurvePrevious});

In [0]:
await DataSource.UpdateAsync(new[]{partition, previousPeriodPartition});
await DataSource.UpdateAsync(new[]{partitionReportingNode});

In [0]:
Workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());

# Test

In [0]:
public async Task<ActivityLog> TestValidation(string inputFile,  List<string> errorBms){
    var ws = Workspace.CreateNew();
    ws.InitializeFrom(DataSource);
    Activity.Start();
    var log = await Import.FromString(inputFile).WithFormat(ImportFormats.DataNodeParameter).WithTarget(ws).ExecuteAsync();
    log.Errors.Count().Should().Be(errorBms.Count());
    errorBms.Intersect(log.Errors.Select(x => x.ToString().Substring(0,x.ToString().Length-2).Substring(40)).ToArray()).Count().Should().Be(errorBms.Count());
    return Activity.Finish();
}

# Use Cases

## Invalid DataNode in Single and Inter Parameter

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.9,Monthly,Uniform
DataNodeInvalid0,0.85,Monthly,Uniform

@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DTR1.1,DT1.1,1
DataNodeInvalid1,DTR1.1,1
DTR1.1,DataNodeInvalid2,1";

var errorsBm = new List<string>(){Error.InvalidDataNode.GetMessage("DataNodeInvalid0"),
                                  Error.InvalidDataNode.GetMessage("DataNodeInvalid1"),
                                  Error.InvalidDataNode.GetMessage("DataNodeInvalid2")};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

## Duplicated DataNode

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation
DT1.1,0.9
DT1.1,0.9

@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DTR1.1,DT1.1,1
DT1.1,DTR1.1,1
";
var errorsBm = new List<string>(){Error.DuplicateSingleDataNode.GetMessage("DT1.1"),
                                  Error.DuplicateInterDataNode.GetMessage("DT1.1","DTR1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

## Single Data Node without Premium Allocation

In [0]:
var minimalParametersScenario = @"
@@Main
ReportingNode,Year,Month
CH,2020,12
@@SingleDataNodeParameter
DataNode
DT1.1
@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DT1.1,DTR1.1,0.62";

In [0]:
var log = await Import.FromString(minimalParametersScenario).WithFormat(ImportFormats.DataNodeParameter).WithTarget(DataSource).ExecuteAsync();
log.Status.Should().Be(ActivityLogStatus.Succeeded);

## Reinsurance Coverage

In [0]:
var inputFile = 
@"
@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,
DT1.1,0.85,
@@InterDataNodeParameter
DataNode,LinkedDataNode,ReinsuranceCoverage
DT1.1,DT1.1,1
";
var errorsBm = new List<string>(){Error.ReinsuranceCoverageDataNode.GetMessage("DT1.1","DT1.1")};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

## Interpolation Method

### Wrong value fails

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,Monthly,InvalidEntry
";
var errorsBm = new List<string>(){Error.InvalidInterpolationMethod.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

### Null value passes when interpolation is required 

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,Monthly,,
";
var errorsBm = new List<string>(){};//Get(Error.InvalidInterpolationMethod, "DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

### Null value is not accepted when interpolation is required

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,Yearly,,
";
var errorsBm = new List<string>(){Error.InvalidInterpolationMethod.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

## CashFlow Periodicity

### Wrong Value

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,A,,
";
var errorsBm = new List<string>(){Error.InvalidCashFlowPeriodicity.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

### Null value not allowed
#### Case 1 : with a valid interpolation 

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,,Uniform,
";
var errorsBm = new List<string>(){Error.InvalidCashFlowPeriodicity.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

#### Case 2 : without a valid interpolation 

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,CashFlowPeriodicity,InterpolationMethod
DT1.1,0.85,,,
";
var errorsBm = new List<string>(){Error.InvalidCashFlowPeriodicity.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

## EconomicBasis

### Wrong Value

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,EconomicBasisDriver
DT1.1,0.85,A
";
var errorsBm = new List<string>(){Error.InvalidEconomicBasisDriver.GetMessage("DT1.1"),};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

### Default values

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,EconomicBasisDriver
DT1.1,0.85,
";

In [0]:
var economicBasisDriverByValuationApproach = new Dictionary<(string,string),string>{
     {(ValuationApproaches.BBA, LiabilityTypes.LIC), EconomicBases.L},
     {(ValuationApproaches.BBA, LiabilityTypes.LRC), EconomicBases.L},
     {(ValuationApproaches.VFA, LiabilityTypes.LIC), EconomicBases.C},
     {(ValuationApproaches.VFA, LiabilityTypes.LRC), EconomicBases.C},
     {(ValuationApproaches.PAA, LiabilityTypes.LIC), EconomicBases.C},
     {(ValuationApproaches.PAA, LiabilityTypes.LRC), EconomicBases.N},
    };

In [0]:
public async Task<bool> CheckDefaultEbDriver((string va, string lt) key, string eb){
    var ws = Workspace.CreateNew();
    ws.InitializeFrom(DataSource);
    ws.InitializeFrom(DataSource);
    await ws.DeleteAsync(ws.Query<GroupOfInsuranceContract>());
    await ws.UpdateAsync<GroupOfInsuranceContract>(new [] {dt11 with {ValuationApproach = key.va, LiabilityType = key.lt}});
    
    var log = await Import.FromString(inputFile).WithFormat(ImportFormats.DataNodeParameter).WithTarget(ws).ExecuteAsync();
    log.Status.Should().Be(ActivityLogStatus.Succeeded);
    return (await ws.Query<SingleDataNodeParameter>().ToArrayAsync()).Single().EconomicBasisDriver == eb;
}

In [0]:
var comparison = new Dictionary<(string,string),bool>();
foreach (var kvp in economicBasisDriverByValuationApproach)
    comparison[kvp.Key] = await CheckDefaultEbDriver(kvp.Key, kvp.Value);

In [0]:
comparison.All(x => x.Value).Should().BeTrue();

## Release Pattern

### Wrong Value

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,ReleasePattern0,ReleasePattern1
DT1.1,0.85,InvalidValue0,InvalidValue1
";
var errorsBm = new List<string>(){Error.ParsingInvalidOrScientificValue.GetMessage("InvalidValue0"), Error.ParsingInvalidOrScientificValue.GetMessage("InvalidValue1")};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity

### Valid or Null Value 

In [0]:
var inputFile = 
@"@@Main
ReportingNode,Year,Month
CH,2020,12

@@SingleDataNodeParameter
DataNode,PremiumAllocation,ReleasePattern0,ReleasePattern1
DT1.1,0.1,1,2
DTR1.1,0.85
";
var errorsBm = new List<string>(){};

In [0]:
var activity = await TestValidation(inputFile, errorsBm);
activity